# Sentiment Classification

Material by Andrew Trask

- **Twitter**: @iamtrask
- **Blog**: http://iamtrask.github.io

Edits & solution by Andrei Dyomin

 - **GitHub**: https://github.com/adyomin

# Project 3: Building a Neural Network

- Start with your neural network from the last chapter
- 3 layer neural network
- no non-linearity in hidden layer
- use our functions to create the training data
- create a "pre_process_data" function to create vocabulary for our training data generating functions
- modify "train" to train over the entire corpus

In [1]:
import sys
nn_path = '/Users/adyomin/Yandex.Disk.localized/Projects/Cornerstone'
sys.path.append(nn_path)
import network as nn
from collections import Counter
import numpy as np
import re

f = open('./data/reviews.txt','r')
reviews = [line.strip('\n') for line in f]
f.close()

f = open('./data/labels.txt','r')
text_labels = [line.strip('\n').upper() for line in f]
f.close()

f = open('./data/full_stop_list.txt','r')
full_stop_list = [line.strip('\n') for line in f]
f.close()

total_counts = Counter()
for review in reviews:
        total_counts.update(re.findall('\w{3,}', review.lower()))
for word in full_stop_list:
    del total_counts[word]

vocab = set(total_counts.keys())
word2index = Counter()
for i, word in enumerate(vocab):
    word2index[word] = i

batch_size = 512
eta = 0.01
n_records = len(text_labels)
n_features = len(vocab)
features = np.zeros(shape=(batch_size, n_features))
labels = np.zeros(shape=(batch_size, 1))
nn_model = nn.Network((73297, 1024, 1))

In [2]:
for epoch in range(1):
    for i in range(0, n_records, batch_size):
        features *= 0
        labels *= 0
        for j, review in enumerate(reviews[i:i + batch_size]):
            for word in review.split():
                if word in vocab:
                    features[j, word2index[word]] += 1
        for k, label in enumerate(text_labels[i:i + batch_size]):
            if label == 'POSITIVE':
                labels[k, 0] = 1
            else:
                labels[k, 0] = 0
        nn_model.train_single_loop(features, labels, batch_size, eta)
        percentage = epoch/10*100
        t_loss = nn_model.evaluate(x=features, y=labels)
        print('Progress: {0:.2f}%, Training loss: {1:.4f}'.format(percentage,
                                                              t_loss))

Progress: 0.00%, Training loss: 0.7024
Progress: 0.00%, Training loss: 1.3592
Progress: 0.00%, Training loss: 2.9606
Progress: 0.00%, Training loss: 6.8551
Progress: 0.00%, Training loss: 16.3839
Progress: 0.00%, Training loss: 39.4978
Progress: 0.00%, Training loss: 95.7279
Progress: 0.00%, Training loss: 232.0319
Progress: 0.00%, Training loss: 558.5185
Progress: 0.00%, Training loss: 1313.5986
Progress: 0.00%, Training loss: 2975.2899
Progress: 0.00%, Training loss: 5867.2693
Progress: 0.00%, Training loss: 10012.2128
Progress: 0.00%, Training loss: 8613.4402
Progress: 0.00%, Training loss: 4980.4903
Progress: 0.00%, Training loss: 1490.3417
Progress: 0.00%, Training loss: 173.1248
Progress: 0.00%, Training loss: 22.6491
Progress: 0.00%, Training loss: 3.8263
Progress: 0.00%, Training loss: 0.7670
Progress: 0.00%, Training loss: 0.3330
Progress: 0.00%, Training loss: 0.2748
Progress: 0.00%, Training loss: 0.2565
Progress: 0.00%, Training loss: 0.2637
Progress: 0.00%, Training loss: 